# Worksheet 4 — k-Nearest Neighbors (KNN) Classification from Scratch

**Dataset:** `diabetes.csv`

## Objective
Perform classification using k-Nearest Neighbors (KNN) algorithm implemented from scratch (without scikit-learn).

## Submission Instructions
- Submit a single notebook containing:
  1. Clean and well-documented code.
  2. Outputs and visualizations.
  3. Detailed explanations and analysis for all steps.
- Ensure all cells are executed before submission.

---

## Problems Overview

**Problem 1:** Implement KNN from scratch, handle missing data, evaluate on diabetes dataset  
**Problem 2:** Compare performance on original vs scaled features  
**Problem 3:** Experiment with different k values (1-15), plot accuracy and time  
**Problem 4:** Discuss challenges and optimization strategies for KNN

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from collections import Counter
from typing import List, Tuple

# Configure display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)
np.set_printoptions(precision=4, suppress=True)

%matplotlib inline
sns.set_style('whitegrid')

print("✓ Libraries imported successfully!")

# Problem 1 — KNN Classification from Scratch

## Step 1: Load the Dataset and Perform EDA

In [ ]:
# Load the dataset
DATA_PATH = 'diabetes.csv'

try:
    df = pd.read_csv(DATA_PATH)
    print(f"✓ Dataset loaded successfully!")
    print(f"Shape: {df.shape}")
except FileNotFoundError as e:
    raise FileNotFoundError(
        f"Could not find {DATA_PATH!r}. Please place diabetes.csv in the same folder as this notebook."
    ) from e

# Display first few rows
print("\n" + "="*80)
print("FIRST 5 ROWS:")
print("="*80)
display(df.head())

# Display last few rows
print("\n" + "="*80)
print("LAST 5 ROWS:")
print("="*80)
display(df.tail())

In [ ]:
# Perform Exploratory Data Analysis (EDA)

print("="*80)
print("DATASET INFORMATION:")
print("="*80)
df.info()

print("\n" + "="*80)
print("DATA TYPES:")
print("="*80)
print(df.dtypes)

print("\n" + "="*80)
print("MISSING VALUES:")
print("="*80)
print(df.isnull().sum())

print("\n" + "="*80)
print("SUMMARY STATISTICS:")
print("="*80)
display(df.describe())

In [ ]:
# Check target variable distribution
target_col = 'Outcome' if 'Outcome' in df.columns else df.columns[-1]
print("="*80)
print(f"TARGET VARIABLE: {target_col}")
print("="*80)
print(df[target_col].value_counts())
print(f"\nClass distribution:")
print(df[target_col].value_counts(normalize=True))

# Visualize target distribution
plt.figure(figsize=(8, 5))
df[target_col].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title(f'Distribution of {target_col}')
plt.xlabel(target_col)
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
numeric_cols = df.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 1:
    plt.figure(figsize=(10, 8))
    corr_matrix = df[numeric_cols].corr()
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
                square=True, linewidths=0.5)
    plt.title('Correlation Heatmap')
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*80)
    print("TOP CORRELATIONS WITH TARGET:")
    print("="*80)
    if target_col in corr_matrix.columns:
        target_corr = corr_matrix[target_col].drop(target_col).sort_values(ascending=False)
        print(target_corr)

## Step 2: Handle Missing Data

For the diabetes dataset, some features may have 0 values that represent missing data (e.g., Glucose, BloodPressure, BMI cannot realistically be 0). We'll handle these appropriately.

In [ ]:
# Handle missing data
df_clean = df.copy()

# Columns that shouldn't have 0 values in diabetes dataset
zero_as_missing = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
present_cols = [col for col in zero_as_missing if col in df_clean.columns]

print("="*80)
print("HANDLING MISSING DATA:")
print("="*80)

if present_cols:
    print(f"\nReplacing 0 with NaN in: {present_cols}")
    for col in present_cols:
        # Count zeros before replacement
        zero_count = (df_clean[col] == 0).sum()
        if zero_count > 0:
            print(f"  {col}: {zero_count} zeros found")
            df_clean.loc[df_clean[col] == 0, col] = np.nan

# Check missing values
print("\n" + "="*80)
print("MISSING VALUES AFTER ZERO REPLACEMENT:")
print("="*80)
missing_counts = df_clean.isnull().sum()
print(missing_counts[missing_counts > 0])

# Impute missing values with median
numeric_features = df_clean.select_dtypes(include=[np.number]).columns.drop(target_col, errors='ignore')

print("\n" + "="*80)
print("IMPUTING MISSING VALUES (using median):")
print("="*80)
for col in numeric_features:
    if df_clean[col].isnull().any():
        median_val = df_clean[col].median()
        df_clean[col].fillna(median_val, inplace=True)
        print(f"  {col}: imputed with median = {median_val:.2f}")

# Drop rows with missing target
if df_clean[target_col].isnull().any():
    print(f"\nDropping {df_clean[target_col].isnull().sum()} rows with missing target values")
    df_clean = df_clean.dropna(subset=[target_col])

print(f"\n✓ Final dataset shape after cleaning: {df_clean.shape}")
print(f"✓ No missing values remaining: {df_clean.isnull().sum().sum() == 0}")

## Step 3: Feature Engineering

Separate features (X) and target (y), then perform train-test split from scratch using 70-30 ratio.

In [ ]:
# Separate features and target
X = df_clean.drop(columns=[target_col]).select_dtypes(include=[np.number]).values
y = df_clean[target_col].values

# Ensure y is integer type
if not np.issubdtype(y.dtype, np.integer):
    if np.all(np.mod(y, 1) == 0):
        y = y.astype(int)
    else:
        # Encode non-integer labels
        unique_labels = np.unique(y)
        label_map = {label: idx for idx, label in enumerate(unique_labels)}
        y = np.array([label_map[label] for label in y])

print("="*80)
print("FEATURE MATRIX AND TARGET:")
print("="*80)
print(f"X shape: {X.shape} (samples × features)")
print(f"y shape: {y.shape} (samples)")
print(f"Unique classes: {np.unique(y)}")
print(f"Class counts: {dict(zip(*np.unique(y, return_counts=True)))}")

In [ ]:
# Train-test split from scratch (70-30 ratio)
def train_test_split_scratch(X, y, test_ratio=0.3, random_state=42):
    """
    Split data into training and test sets from scratch.
    
    Parameters:
        X: Feature matrix
        y: Target vector
        test_ratio: Proportion for test set (default 0.3 = 30%)
        random_state: Random seed for reproducibility
    
    Returns:
        X_train, X_test, y_train, y_test
    """
    np.random.seed(random_state)
    n_samples = X.shape[0]
    
    # Shuffle indices
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    # Calculate split point
    test_size = int(n_samples * test_ratio)
    train_size = n_samples - test_size
    
    # Split
    train_idx = indices[:train_size]
    test_idx = indices[train_size:]
    
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

# Perform split
X_train, X_test, y_train, y_test = train_test_split_scratch(X, y, test_ratio=0.3, random_state=42)

print("\n" + "="*80)
print("TRAIN-TEST SPLIT (70-30):")
print("="*80)
print(f"Training set: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set:     X_test={X_test.shape}, y_test={y_test.shape}")
print(f"\nTrain class distribution: {dict(zip(*np.unique(y_train, return_counts=True)))}")
print(f"Test class distribution:  {dict(zip(*np.unique(y_test, return_counts=True)))}")

## Step 4: Implement KNN from Scratch

We'll implement:
1. **Euclidean distance** calculation
2. **Predict single query** - find k nearest neighbors and vote
3. **Predict all test samples** - batch prediction
4. **Accuracy metric** - evaluate performance

In [ ]:
def euclidean_distance(point1, point2):
    """
    Calculate Euclidean distance between two points.
    
    Parameters:
        point1: First point (numpy array)
        point2: Second point (numpy array)
    
    Returns:
        distance: Euclidean distance
    """
    return np.sqrt(np.sum((point1 - point2) ** 2))


def knn_predict_single(X_train, y_train, query_point, k):
    """
    Predict class for a single query point using KNN.
    
    Parameters:
        X_train: Training feature matrix
        y_train: Training labels
        query_point: Single test point to classify
        k: Number of nearest neighbors
    
    Returns:
        predicted_class: Predicted class label
    """
    # Calculate distances from query point to all training points
    distances = []
    for i in range(len(X_train)):
        dist = euclidean_distance(query_point, X_train[i])
        distances.append((dist, y_train[i]))
    
    # Sort by distance and get k nearest neighbors
    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]
    
    # Get labels of k nearest neighbors
    k_labels = [label for _, label in k_nearest]
    
    # Majority vote (use Counter for tie-breaking)
    vote_counts = Counter(k_labels)
    # If tie, choose the class that appears first among nearest neighbors
    predicted_class = vote_counts.most_common(1)[0][0]
    
    return predicted_class


def knn_predict(X_train, y_train, X_test, k):
    """
    Predict classes for all test samples.
    
    Parameters:
        X_train: Training feature matrix
        y_train: Training labels
        X_test: Test feature matrix
        k: Number of nearest neighbors
    
    Returns:
        predictions: Array of predicted labels
    """
    predictions = []
    for test_point in X_test:
        pred = knn_predict_single(X_train, y_train, test_point, k)
        predictions.append(pred)
    return np.array(predictions)


def accuracy_score(y_true, y_pred):
    """
    Calculate accuracy.
    
    Parameters:
        y_true: True labels
        y_pred: Predicted labels
    
    Returns:
        accuracy: Accuracy score (0 to 1)
    """
    return np.mean(y_true == y_pred)


print("✓ KNN functions implemented successfully!")

In [ ]:
# Test KNN with k=5 on original (unscaled) data
k_default = 5

print("="*80)
print(f"TESTING KNN WITH k={k_default} (Original/Unscaled Data)")
print("="*80)

t_start = time.time()
y_pred_original = knn_predict(X_train, y_train, X_test, k=k_default)
t_end = time.time()

accuracy_original = accuracy_score(y_test, y_pred_original)
time_original = t_end - t_start

print(f"\n✓ Prediction completed!")
print(f"  Accuracy: {accuracy_original:.4f} ({accuracy_original*100:.2f}%)")
print(f"  Prediction time: {time_original:.4f} seconds")
print(f"  Predictions shape: {y_pred_original.shape}")

# Show confusion information
print(f"\nSample predictions (first 10):")
print(f"  True:      {y_test[:10]}")
print(f"  Predicted: {y_pred_original[:10]}")

# Calculate per-class accuracy
for class_label in np.unique(y_test):
    mask = y_test == class_label
    class_acc = accuracy_score(y_test[mask], y_pred_original[mask])
    print(f"  Class {class_label} accuracy: {class_acc:.4f}")

# Problem 2 — Experimentation with Scaling

## Compare Original vs Scaled Features

KNN is sensitive to feature scales because it uses distance metrics. Features with larger ranges can dominate the distance calculation.

In [ ]:
# Implement standardization (z-score normalization) from scratch
def standardize_features(X_train, X_test):
    """
    Standardize features using z-score normalization.
    Fit on training data, transform both train and test.
    
    Parameters:
        X_train: Training feature matrix
        X_test: Test feature matrix
    
    Returns:
        X_train_scaled, X_test_scaled: Standardized matrices
    """
    # Calculate mean and std from training data only
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)
    
    # Avoid division by zero
    std_safe = np.where(std == 0, 1, std)
    
    # Standardize
    X_train_scaled = (X_train - mean) / std_safe
    X_test_scaled = (X_test - mean) / std_safe
    
    return X_train_scaled, X_test_scaled

# Scale the features
X_train_scaled, X_test_scaled = standardize_features(X_train, X_test)

print("="*80)
print("FEATURE SCALING (Standardization):")
print("="*80)
print(f"Original X_train - Mean: {X_train.mean(axis=0)[:3]}... ")
print(f"Original X_train - Std:  {X_train.std(axis=0)[:3]}...")
print(f"\nScaled X_train - Mean: {X_train_scaled.mean(axis=0)[:3]}...")
print(f"Scaled X_train - Std:  {X_train_scaled.std(axis=0)[:3]}...")
print(f"\n✓ Features scaled successfully!")

In [ ]:
# Test KNN on scaled data
print("="*80)
print(f"TESTING KNN WITH k={k_default} (Scaled Data)")
print("="*80)

t_start = time.time()
y_pred_scaled = knn_predict(X_train_scaled, y_train, X_test_scaled, k=k_default)
t_end = time.time()

accuracy_scaled = accuracy_score(y_test, y_pred_scaled)
time_scaled = t_end - t_start

print(f"\n✓ Prediction completed!")
print(f"  Accuracy: {accuracy_scaled:.4f} ({accuracy_scaled*100:.2f}%)")
print(f"  Prediction time: {time_scaled:.4f} seconds")

# Per-class accuracy
for class_label in np.unique(y_test):
    mask = y_test == class_label
    class_acc = accuracy_score(y_test[mask], y_pred_scaled[mask])
    print(f"  Class {class_label} accuracy: {class_acc:.4f}")

## Comparative Analysis: Original vs Scaled

In [ ]:
# Compare results
comparison = pd.DataFrame({
    'Dataset': ['Original', 'Scaled'],
    'Accuracy': [accuracy_original, accuracy_scaled],
    'Prediction Time (s)': [time_original, time_scaled],
    'Improvement': ['Baseline', f"{((accuracy_scaled - accuracy_original) / accuracy_original * 100):+.2f}%"]
})

print("="*80)
print("COMPARISON: ORIGINAL vs SCALED")
print("="*80)
display(comparison)

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Accuracy comparison
axes[0].bar(['Original', 'Scaled'], [accuracy_original, accuracy_scaled], 
            color=['skyblue', 'lightcoral'])
axes[0].set_ylabel('Accuracy')
axes[0].set_title(f'Accuracy Comparison (k={k_default})')
axes[0].set_ylim([0, 1])
axes[0].grid(axis='y', alpha=0.3)
for i, v in enumerate([accuracy_original, accuracy_scaled]):
    axes[0].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')

# Time comparison
axes[1].bar(['Original', 'Scaled'], [time_original, time_scaled], 
            color=['skyblue', 'lightcoral'])
axes[1].set_ylabel('Time (seconds)')
axes[1].set_title('Prediction Time Comparison')
axes[1].grid(axis='y', alpha=0.3)
for i, v in enumerate([time_original, time_scaled]):
    axes[1].text(i, v + 0.02, f'{v:.3f}s', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("DISCUSSION:")
print("="*80)
print("""
**Why Scaling Matters for KNN:**

1. **Distance Metric Sensitivity:** KNN uses Euclidean distance. Features with larger 
   numeric ranges (e.g., Glucose: 0-200) dominate over smaller ranges (e.g., Pregnancies: 0-17).

2. **Equal Feature Contribution:** Scaling ensures all features contribute equally to 
   distance calculations, preventing bias toward high-magnitude features.

3. **Performance Impact:** 
   - If accuracy improves: Features were on different scales, scaling helped.
   - If accuracy stays similar: Features may already be comparable, or k needs tuning.
   - If accuracy drops: Scaling may amplify noise in weak features.

4. **Computational Cost:** Scaling doesn't significantly affect prediction time for 
   our implementation (dominated by distance calculations).
""")

# Problem 3 — Experimentation with k Values (1 to 15)

Test KNN performance for k = 1, 2, 3, ..., 15 on both original and scaled datasets.
Record accuracy and prediction time for each k.

In [ ]:
# Experiment with k from 1 to 15
k_values = list(range(1, 16))

results_original_k = []
results_scaled_k = []

print("="*80)
print("EXPERIMENTING WITH DIFFERENT k VALUES (1-15)")
print("="*80)

for k in k_values:
    print(f"\nTesting k={k}...")
    
    # Original data
    t_start = time.time()
    y_pred_orig = knn_predict(X_train, y_train, X_test, k=k)
    t_end = time.time()
    acc_orig = accuracy_score(y_test, y_pred_orig)
    time_orig = t_end - t_start
    results_original_k.append({
        'k': k,
        'accuracy': acc_orig,
        'time': time_orig
    })
    
    # Scaled data
    t_start = time.time()
    y_pred_scal = knn_predict(X_train_scaled, y_train, X_test_scaled, k=k)
    t_end = time.time()
    acc_scal = accuracy_score(y_test, y_pred_scal)
    time_scal = t_end - t_start
    results_scaled_k.append({
        'k': k,
        'accuracy': acc_scal,
        'time': time_scal
    })
    
    print(f"  Original - Accuracy: {acc_orig:.4f}, Time: {time_orig:.3f}s")
    print(f"  Scaled   - Accuracy: {acc_scal:.4f}, Time: {time_scal:.3f}s")

# Convert to DataFrames
df_results_orig = pd.DataFrame(results_original_k)
df_results_scal = pd.DataFrame(results_scaled_k)

print("\n" + "="*80)
print("RESULTS SUMMARY:")
print("="*80)
print("\nOriginal Dataset:")
display(df_results_orig)

print("\nScaled Dataset:")
display(df_results_scal)

## Visualize Results: k vs Accuracy and k vs Time

In [ ]:
# Plot k vs Accuracy and k vs Time
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: k vs Accuracy
axes[0].plot(df_results_orig['k'], df_results_orig['accuracy'], 
             marker='o', linewidth=2, label='Original', color='blue')
axes[0].plot(df_results_scal['k'], df_results_scal['accuracy'], 
             marker='s', linewidth=2, label='Scaled', color='red')
axes[0].set_xlabel('k (Number of Neighbors)', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('k vs Accuracy', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks(k_values)

# Plot 2: k vs Time
axes[1].plot(df_results_orig['k'], df_results_orig['time'], 
             marker='o', linewidth=2, label='Original', color='blue')
axes[1].plot(df_results_scal['k'], df_results_scal['time'], 
             marker='s', linewidth=2, label='Scaled', color='red')
axes[1].set_xlabel('k (Number of Neighbors)', fontsize=12)
axes[1].set_ylabel('Prediction Time (seconds)', fontsize=12)
axes[1].set_title('k vs Prediction Time', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].set_xticks(k_values)

plt.tight_layout()
plt.show()

## Analysis: Identify Optimal k

In [ ]:
# Find optimal k for each dataset
best_k_orig = df_results_orig.loc[df_results_orig['accuracy'].idxmax()]
best_k_scal = df_results_scal.loc[df_results_scal['accuracy'].idxmax()]

print("="*80)
print("OPTIMAL k VALUES:")
print("="*80)
print(f"\nOriginal Dataset:")
print(f"  Best k: {int(best_k_orig['k'])}")
print(f"  Accuracy: {best_k_orig['accuracy']:.4f}")
print(f"  Time: {best_k_orig['time']:.4f}s")

print(f"\nScaled Dataset:")
print(f"  Best k: {int(best_k_scal['k'])}")
print(f"  Accuracy: {best_k_scal['accuracy']:.4f}")
print(f"  Time: {best_k_scal['time']:.4f}s")

print("\n" + "="*80)
print("DISCUSSION:")
print("="*80)
print("""
**How k Affects Accuracy and Computational Cost:**

1. **Small k (k=1, 2, 3):**
   - **Accuracy:** Can be high but sensitive to noise and outliers. Overfitting risk.
   - **Decision Boundary:** Complex, non-smooth boundaries.
   - **Variance:** High variance, low bias.

2. **Moderate k (k=5-10):**
   - **Accuracy:** Often provides good balance.
   - **Decision Boundary:** Smoother, more generalized.
   - **Bias-Variance:** Balanced trade-off.

3. **Large k (k>10):**
   - **Accuracy:** May decrease if too large (underfitting).
   - **Decision Boundary:** Very smooth, may miss local patterns.
   - **Variance:** Low variance, high bias.

4. **Computational Cost:**
   - **Distance Calculation:** Dominant cost (O(n*d) per query, where n=training size, d=features).
   - **k Impact:** Sorting/voting adds O(k log k), but negligible compared to distance computation.
   - **Our Results:** Time relatively constant across k values (dominated by distance calculations).

5. **Optimal k Selection:**
   - Choose k with highest validation/test accuracy.
   - Prefer odd k for binary classification (avoids ties).
   - Use cross-validation for robust selection.
   - Consider both accuracy and computational constraints.
""")

# Problem 4 — Additional Questions (Optional but Recommended)

## Challenges of KNN for Large Datasets and High-Dimensional Data

### 1. Challenges of Using KNN

#### **For Large Datasets:**

**a) Computational Complexity:**
- **Training:** O(1) - lazy learning (no explicit training phase)
- **Prediction:** O(n × d) per query, where:
  - n = number of training samples
  - d = number of features
- **Problem:** Linear scaling with dataset size makes prediction slow for large n

**b) Memory Requirements:**
- Must store entire training dataset in memory
- Space complexity: O(n × d)
- **Problem:** Memory consumption scales linearly with data size

**c) Prediction Latency:**
- Each query requires computing distance to ALL training points
- Real-time applications become impractical with large datasets
- **Example:** For 1 million samples, each prediction requires 1 million distance calculations

---

#### **For High-Dimensional Data:**

**a) Curse of Dimensionality:**
- As dimensions increase, data becomes sparse in the feature space
- **Problem:** All points become "far" from each other, making "nearest" meaningless
- Distance concentration: max_dist/min_dist → 1 as d → ∞

**b) Irrelevant Features:**
- Many features may be noise or irrelevant
- **Problem:** Distance calculations are polluted by non-informative dimensions
- Equal weighting of all features can degrade performance

**c) Computational Burden:**
- Distance calculation cost grows with dimensions: O(d)
- High memory usage: O(n × d)
- **Problem:** Both time and space complexity increase

**d) Loss of Local Structure:**
- Nearest neighbors in high dimensions may not be meaningfully "near"
- **Problem:** KNN's fundamental assumption breaks down

---

### 2. Strategies to Improve KNN Efficiency

#### **A. Spatial Indexing Structures**

**1. KD-Tree (K-Dimensional Tree):**
- **How it works:** Binary tree that recursively partitions space along different dimensions
- **Complexity:** O(log n) average query time for low dimensions
- **Limitation:** Performance degrades to O(n) for d > 20 (curse of dimensionality)
- **Best for:** Low-dimensional data (d < 10-20)

**2. Ball Tree:**
- **How it works:** Organizes data into nested hyperspheres
- **Complexity:** O(log n) query time, more robust than KD-tree for higher dimensions
- **Advantage:** Better than KD-tree for d > 10
- **Best for:** Moderate-dimensional data

**3. Cover Tree:**
- **How it works:** Hierarchical data structure with distance-based levels
- **Complexity:** O(log n) query time with theoretical guarantees
- **Advantage:** Adapts to intrinsic dimensionality of data

---

#### **B. Approximate Nearest Neighbors (ANN)**

Trade exact neighbors for speed with controlled accuracy loss:

**1. Locality-Sensitive Hashing (LSH):**
- **How it works:** Hash similar items to same buckets using special hash functions
- **Complexity:** O(1) to O(log n) query time
- **Advantage:** Scales to very large datasets
- **Use case:** Image retrieval, duplicate detection

**2. HNSW (Hierarchical Navigable Small World):**
- **How it works:** Multi-layer graph structure for approximate search
- **Complexity:** O(log n) query time
- **Advantage:** State-of-the-art speed/accuracy trade-off
- **Use case:** Modern vector databases, semantic search

**3. Product Quantization:**
- **How it works:** Compress vectors using learned codebooks
- **Advantage:** Reduces memory footprint dramatically
- **Use case:** Large-scale image/video search

---

#### **C. Dimensionality Reduction**

**1. Principal Component Analysis (PCA):**
- **How it works:** Project data onto top k principal components
- **Advantage:** Removes correlated/redundant features, reduces noise
- **Typical:** Reduce to 50-100 dimensions retaining 90-95% variance

**2. t-SNE / UMAP:**
- **How it works:** Non-linear dimensionality reduction preserving local structure
- **Use case:** Visualization, preprocessing for KNN

**3. Autoencoders:**
- **How it works:** Neural network learns compressed representation
- **Advantage:** Learns task-specific low-dimensional features

---

#### **D. Feature Selection**

**1. Filter Methods:**
- Select features based on statistical tests (correlation, mutual information)
- Remove irrelevant/redundant features before KNN

**2. Wrapper Methods:**
- Use KNN performance to guide feature selection
- Forward/backward selection with cross-validation

**3. Embedded Methods:**
- L1 regularization (Lasso) to identify important features
- Tree-based feature importance

---

#### **E. Algorithm-Level Optimizations**

**1. Distance Metric Learning:**
- Learn a custom distance metric from data (e.g., Mahalanobis distance)
- Weight features by importance
- **Advantage:** Improves accuracy without changing k

**2. Prototype Selection / Data Reduction:**
- **Condensed Nearest Neighbor (CNN):** Keep only boundary samples
- **Edited Nearest Neighbor (ENN):** Remove noisy samples
- **Advantage:** Reduce n, speeding up prediction

**3. Parallel/GPU Computation:**
- Parallelize distance calculations across multiple cores/GPUs
- Use vectorized operations (NumPy, PyTorch, JAX)
- **Advantage:** Linear speedup with hardware

**4. Early Stopping:**
- Stop searching once k neighbors are found within a distance threshold
- Use branch-and-bound pruning in tree structures

---

#### **F. Hybrid Approaches**

**1. Ensemble Methods:**
- Combine KNN with other classifiers (Random Forest, SVM)
- Use KNN locally and global model for final prediction

**2. Preprocessing Pipelines:**
- Combine multiple strategies:
  - Feature scaling → PCA → KD-Tree → KNN
  - Feature selection → LSH → Approximate KNN

---

### **Summary: Practical Recommendations**

| Dataset Characteristics | Recommended Strategy |
|------------------------|---------------------|
| **n < 10,000, d < 20** | KD-Tree or Ball Tree |
| **n > 100,000, d < 50** | HNSW or LSH (ANN) |
| **d > 50** | PCA → reduce to d < 30 → KD-Tree |
| **d > 100** | Feature selection + PCA + ANN |
| **Real-time queries** | Pre-build index (HNSW) + GPU acceleration |
| **Memory constrained** | Product quantization + prototype selection |

**Key Takeaway:** The best strategy depends on:
- Dataset size (n)
- Dimensionality (d)
- Latency requirements
- Accuracy tolerance
- Available hardware